In [1]:
import glob
import sys
import torch

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
from vandy_taggers.modules.models.part import ParticleTransformer, PartTrainer
from vandy_taggers.utils.data_loader import DataLoader

In [4]:
OUT_CONVERTED_PATH = "../../../Datalake/converted/*train*"
BS = 32
model = ParticleTransformer(num_classes = 6,
                 num_enc = 8,
                 num_head = 8,
                 embed_dim = 128,
                 cpf_dim = 16,
                 npf_dim = 8,
                 vtx_dim = 14,
                 for_inference = False,)
model.to(device);

In [5]:
train_loader = DataLoader(OUT_CONVERTED_PATH, BS).get_loader()
val_loader = DataLoader(OUT_CONVERTED_PATH, BS).get_loader()

In [6]:
num_epochs = 1
lr_epochs = max(1, int(num_epochs * 0.3))
lr_rate = 0.01 ** (1.0 / lr_epochs)
mil = list(range(num_epochs - lr_epochs, num_epochs))
print(lr_rate)
print(mil)

0.01
[0]


In [7]:
from vandy_taggers.utils.pt_ranger import Ranger


In [8]:
optimizer = Ranger(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=mil, gamma=lr_rate)

In [9]:
trainer = PartTrainer(
    optimizer=optimizer,
    scheduler=scheduler,
    model=model,
    device=device)

In [10]:
loss_f = torch.nn.CrossEntropyLoss()

In [11]:
def cross_entropy_one_hot(input, target):
    _, labels = target.max(dim=1)
    return torch.nn.CrossEntropyLoss()(input, labels)


In [12]:
trainer.train(num_epochs, train_loader, val_loader, path="/home/jose/Documents/WORKS/CERN/InProgress/OwnArch/saved_models")

  0%|          | 0/1 [00:00<?, ?it/s]

Train files 102
Val files 102
Epoch:  0
